In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import shapefile as sf
from pyproj import Proj
from censusgeocode import CensusGeocode

import datetime

%matplotlib inline

In [213]:
police_calls = pd.read_csv('pd_calls_for_service_2016_datasd.csv')

In [214]:
police_calls.columns.values

array(['incident_num', 'date_time', 'day', 'stno', 'stdir1', 'street',
       'streettype', 'stdir2', 'stname2', 'sttype2', 'call_type',
       'disposition', 'beat', 'priority'], dtype=object)

In [215]:
arrest_index = np.where(police_calls['disposition'].str.contains('A')==True)[0]

In [219]:
police_arrests = police_calls.loc[arrest_index]
police_arrests.reset_index(drop=True, inplace=True)

In [220]:
police_arrests.loc[np.where(pd.isnull(police_arrests['stdir1'])==True)[0], 'stdir1'] = ''
police_arrests.loc[np.where(pd.isnull(police_arrests['streettype'])==True)[0], 'streettype'] = ''

In [221]:
street_fix_index = np.where(police_arrests['street'].str.startswith('0')==True)[0]
police_arrests.loc[street_fix_index, 'street'] = police_arrests.loc[street_fix_index, 'street'].replace({'0': ''}, regex=True)

In [110]:
cg = CensusGeocode()

In [222]:
police_arrests['lat'] = None
police_arrests['lon'] = None 
police_arrests['tract'] = None
police_arrests['blockgroup'] = None 
police_arrests['block '] = None

In [ ]:
start = datetime.datetime.now()

print len(police_arrests)

for i in range(0,len(police_arrests)): 
    address1 = (police_arrests.loc[i, 'stno'].astype(str) + ' ' + police_arrests.loc[i, 'stdir1'] 
                + ' ' + police_arrests.loc[i, 'street'] + ' ' 
                + police_arrests.loc[i, 'streettype'])
    temp = cg.address(address1, city='San Diego', state='CA')
    if (temp!=[]): 
        if (len(temp)>0): 
            police_arrests.loc[i, 'lon'] = temp[0]['coordinates']['x']
            police_arrests.loc[i, 'lat'] = temp[0]['coordinates']['y']

            police_arrests.loc[i, 'tract'] = temp[0]['geographies']['2010 Census Blocks'][0]['TRACT']
            police_arrests.loc[i, 'block'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLOCK']
            police_arrests.loc[i, 'blockgroup'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']

        if (i % 100 == 0): 
            finish = datetime.datetime.now()
            print i, 'Time Diff: ', finish-start, 'Finish Time: ', finish
            start = datetime.datetime.now()

        if (i % 500 == 0): 
            police_arrests.to_csv('output_files/police_arrests_prelim.csv', index=False)

44439
0 Time Diff:  0:00:01.548325 Finish Time:  2016-09-17 15:24:15.734926


In [171]:
print i
print address1

752
1000  02ND AV


In [172]:
temp = cg.address(address1, city='San Diego', state='CA')

In [173]:
if (temp!=[]): 
    print 'hello'

In [175]:
temp = cg.address('1000  2ND AV', city='San Diego', state='CA')
len(temp)

1

Organise arrests by tract and block group

In [31]:
police_arrests['arrests_count'] = 1.0

In [28]:
police_arrests['tract'] = police_arrests['tract'].astype(float)
police_arrests['blockgroup'] = police_arrests['blockgroup'].astype(float)

In [39]:
pa_by_bg = police_arrests[['tract', 'blockgroup', 'pa_count']].groupby(['tract','blockgroup'], as_index=False).agg('count')

In [40]:
#cpa_by_bg.to_csv('output_files/police_arrests_bg.csv', index=False)